# Road Vehicles Platooning using Multi-Agent Reinforcement Learning
**Statistical Machine Learning Project**\
 **Prepared by: Charbel ABI HANA**

# Table of Contents


## Intro 

## Reinforcement Learning

## Multi-Agent Reinforcement Learning

## OpenAI GYM

## Highway Environment

### Envrionment Overview

### Action Space

### Observation Space

### Reward Function

### Multi-Agent Layout

### IDQN - Independant Agents - Development